In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import torch
import gc
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
import seaborn as sns

from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Import Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/test.csv")
ss = pd.read_csv('/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/submission.csv')

In [6]:
train_df = train.drop(['id','f2','f35','f44','target'], axis = 1 )
test_df = test.drop(['id','f2','f35','f44'], axis = 1)

# EDA-1 세제곱근 씌워서 파생 변수 창출

In [7]:
train.isnull().sum()[train.isnull().sum() != 0]

Series([], dtype: int64)

In [8]:
features = test_df.columns.tolist()
len(features)

97

In [9]:
for col in tqdm(features):
    train_df[col+'_bin'] = train_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)
    test_df[col+'_bin'] = test_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)

print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")
train_df.head()

100%|██████████| 97/97 [02:30<00:00,  1.55s/it]

train_df: (600000, 194) 
test_df: (540000, 194)


,f0,f1,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f36,f37,f38,f39,f40,f41,...,f60_bin,f61_bin,f62_bin,f63_bin,f64_bin,f65_bin,f66_bin,f67_bin,f68_bin,f69_bin,f70_bin,f71_bin,f72_bin,f73_bin,f74_bin,f75_bin,f76_bin,f77_bin,f78_bin,f79_bin,f80_bin,f81_bin,f82_bin,f83_bin,f84_bin,f85_bin,f86_bin,f87_bin,f88_bin,f89_bin,f90_bin,f91_bin,f92_bin,f93_bin,f94_bin,f95_bin,f96_bin,f97_bin,f98_bin,f99_bin
0,0.106643,3.59437,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.806880,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.33055,0.009397,0.144738,3.05131,1.30362,0.033225,-0.018284,2.748210,-0.009294,-0.036271,-0.049871,0.019484,3.898460,1.138020,3.366880,4.94446,-0.105772,2.11345,3.452230,...,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1
1,0.125021,1.67336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.262150,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.60444,0.103441,0.067638,4.75362,1.85552,-0.181834,0.008359,3.166340,0.011850,0.022292,0.069320,0.117109,0.315276,1.672270,-0.409067,4.95475,0.092358,2.60318,1.954690,...,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1
2,0.036330,1.49747,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.069490,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.68365,0.038180,0.123716,1.11248,3.57166,0.120601,0.082069,2.233520,0.002270,0.045182,0.014405,0.011599,-0.502849,1.417500,1.071350,3.22296,2.122030,3.08216,0.637555,...,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1
3,-0.014077,0.24600,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,1.002680,4.869600,0.058411,2.497850,1.23843,2.348360,0.175475,1.608890,2.02881,0.042086,0.005141,0.076506,1.65122,0.111813,0.121641,0.58912,4.23692,-0.032843,0.058168,0.712927,0.097465,0.072744,0.000324,0.063362,4.063820,0.576572,2.026210,2.96843,1.085670,1.71088,1.371820,...,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,1,1
4,-0.003259,3.71542,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,0.711483,0.769988,0.057555,0.957257,3.71145,5.464350,0.287104,2.616950,1.38403,0.074883,-0.010543,0.109121,2.27602,0.008023,0.045236,4.35954,5.07562,-0.009376,0.528966,4.053350,0.020000,0.106828,0.051307,0.045939,3.402460,1.635960,0.047029,4.01771,0.155748,5.28998,4.118920,...,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1


In [16]:
train_df = pd.concat([train_df, train['target']],axis = 1)

In [17]:
features = test_df.columns.tolist()
print(f"Num features: {len(features)}")

Num features: 194


In [18]:
train_df[features] = train_df[features].astype('float32')
test_df[features] = test_df[features].astype('float32')
print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

train_df: (600000, 195) 
test_df: (540000, 194)


# Model1-DNN

In [12]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Multiply

np.random.seed(42)
tf.random.set_seed(42)

In [13]:
def plot_confusion_matrix(cm, classes):

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', fontweight='bold', pad=15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontweight='bold')
    plt.xlabel('Predicted label', fontweight='bold')
    plt.tight_layout()

In [14]:
def dnn_model1():
    
    x_input = Input(shape=(len(features),))
    
    x1 = Dense(units=384, activation='swish')(x_input)
    x1 = BatchNormalization()(x1)
    x2 = Dropout(rate=0.45)(x1)
    
    x2 = Dense(units=192, activation='swish')(x2)
    x2 = BatchNormalization()(x2)
    x3 = Dropout(rate=0.35)(x2)
    
    x3 = Dense(units=96, activation='swish')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(rate=0.25)(x3)
    
    x4 = Dense(units=192, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    
    x5 = Dense(units=384, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    
    x = Concatenate()([x3, x5])
    x = Dense(units=128, activation='swish')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

model1 = dnn_model1()
model1.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 194)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 384)          74880       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 384)         1536        ['dense[0][0]']                  
 alization)                                                                                       
                                                                                                  
 dropout (Dropout)              (None, 384)          0           ['batch_normalization[0][

In [27]:
def dnn_model2():
    
    x_input = Input(shape=(len(features),))
    
    x1 = Dense(units=196, activation='swish')(x_input)
    x1 = BatchNormalization()(x1)
    x2 = Dropout(rate=0.45)(x1)
    
    x2 = Dense(units=96, activation='swish')(x2)
    x2 = BatchNormalization()(x2)
    x3 = Dropout(rate=0.35)(x2)
    
    x3 = Dense(units=48, activation='swish')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(rate=0.25)(x3)
    
    x4 = Dense(units=96, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    
    x5 = Dense(units=196, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    
    x = Concatenate()([x3, x5])
    x = Dense(units=64, activation='swish')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

model2 = dnn_model2()
model2.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 194)]        0           []                               
                                                                                                  
 dense_96 (Dense)               (None, 196)          38220       ['input_15[0][0]']               
                                                                                                  
 batch_normalization_83 (BatchN  (None, 196)         784         ['dense_96[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 dropout_82 (Dropout)           (None, 196)          0           ['batch_normalization_83[

In [30]:
def dnn_model3():
    
    x_input = Input(shape=(len(features),))
    
    x1 = Dense(units=98, activation='swish')(x_input)
    x1 = BatchNormalization()(x1)
    x2 = Dropout(rate=0.45)(x1)
    
    x2 = Dense(units=48, activation='swish')(x2)
    x2 = BatchNormalization()(x2)
    x3 = Dropout(rate=0.35)(x2)
    
    x3 = Dense(units=24, activation='swish')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(rate=0.25)(x3)
    
    x4 = Dense(units=48, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    
    x5 = Dense(units=98, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    
    x = Concatenate()([x3, x5])
    x = Dense(units=32, activation='swish')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

model3 = dnn_model3()
model3.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 194)]        0           []                               
                                                                                                  
 dense_115 (Dense)              (None, 98)           19110       ['input_18[0][0]']               
                                                                                                  
 batch_normalization_100 (Batch  (None, 98)          392         ['dense_115[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 dropout_98 (Dropout)           (None, 98)           0           ['batch_normalization_100

In [40]:
def dnn_model4():
    
    x_input = Input(shape=(len(features),))
    
    x1 = Dense(units=49, activation='swish')(x_input)
    x1 = BatchNormalization()(x1)
    x2 = Dropout(rate=0.45)(x1)
    
    x2 = Dense(units=24, activation='swish')(x2)
    x2 = BatchNormalization()(x2)
    x3 = Dropout(rate=0.35)(x2)
    
    x3 = Dense(units=12, activation='swish')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(rate=0.25)(x3)
    
    x4 = Dense(units=24, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    
    x5 = Dense(units=49, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    
    x = Concatenate()([x3, x5])
    x = Dense(units=16, activation='swish')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

model4 = dnn_model4()
model4.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_39 (InputLayer)          [(None, 194)]        0           []                               
                                                                                                  
 dense_262 (Dense)              (None, 49)           9555        ['input_39[0][0]']               
                                                                                                  
 batch_normalization_226 (Batch  (None, 49)          196         ['dense_262[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 dropout_224 (Dropout)          (None, 49)           0           ['batch_normalization_226

**dnn_Model1**

In [19]:
### dnn_model1

FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model1 = dnn_model1()
        model1.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model1.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model1 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model1.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model1.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model1, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.7527734954240652



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.7566354546198236



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7574400717789872



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7544944849076469



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.7572055679575798


Seed: 2025 | Aggregate OOF Score: 0.7557024065345116


Aggregate OOF Score: 0.7557838607531482


In [21]:
y_pred_meta = np.mean(y_pred_meta_dnn, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.74      0.73      0.74    296394
           1       0.74      0.75      0.75    303606

    accuracy                           0.74    600000
   macro avg       0.74      0.74      0.74    600000
weighted avg       0.74      0.74      0.74    600000



In [22]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [23]:
submit_df = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df['target'] = y_pred_final_dnn.ravel()
submit_df.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN1_cbrt_Submission.csv", index=False)
submit_df.head()

,id,target
0,600000,0.749574
1,600001,0.714452
2,600002,0.765703
3,600003,0.331450
4,600004,0.675498


# Model2-DNN

In [32]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model2 = dnn_model2()
        model2.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model2.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model2 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model2.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model2.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model2, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn2 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn2 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.7529289020314891



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.7570648995484572



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7577676518594196



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7545757117757181



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.757107022993715


Seed: 2025 | Aggregate OOF Score: 0.755693300100529


Aggregate OOF Score: 0.7558462757248299


In [33]:
y_pred_meta = np.mean(y_pred_meta_dnn2, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.74      0.73      0.74    296394
           1       0.74      0.75      0.75    303606

    accuracy                           0.74    600000
   macro avg       0.74      0.74      0.74    600000
weighted avg       0.74      0.74      0.74    600000



In [34]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [35]:
submit_df2 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df2['target'] = y_pred_final_dnn2.ravel()
submit_df2.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model2_Submission.csv", index=False)
submit_df2.head()

,id,target
0,600000,0.751021
1,600001,0.717366
2,600002,0.764463
3,600003,0.332137
4,600004,0.677350


# Model3-DNN

In [36]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model3 = dnn_model3()
        model3.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model3.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model3 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model3.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model3.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model3, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn3 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn3 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.7528662181188037



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.7572968756847647



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7574156629765715



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7546679919065502



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.7573783840926593


Seed: 2025 | Aggregate OOF Score: 0.7558356254518811


Aggregate OOF Score: 0.7559661443233621


In [37]:
y_pred_meta = np.mean(y_pred_meta_dnn3, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.74      0.73      0.74    296394
           1       0.74      0.75      0.75    303606

    accuracy                           0.74    600000
   macro avg       0.74      0.74      0.74    600000
weighted avg       0.74      0.74      0.74    600000



In [38]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [39]:
submit_df3 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df3['target'] = y_pred_final_dnn3.ravel()
submit_df3.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model3_Submission.csv", index=False)
submit_df3.head()

,id,target
0,600000,0.745709
1,600001,0.721099
2,600002,0.759180
3,600003,0.327330
4,600004,0.682954


Model4-DNN

In [41]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model4 = dnn_model4()
        model4.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model4.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model4 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model4.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model4.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model4, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn4 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn4 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.7526248195107775



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.7574569074045006



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7574831538333812



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7544746577390427



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.7571025741550739


Seed: 2025 | Aggregate OOF Score: 0.7558102357912742


Aggregate OOF Score: 0.7558615202997034


In [42]:
y_pred_meta = np.mean(y_pred_meta_dnn4, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.74      0.73      0.74    296394
           1       0.74      0.75      0.75    303606

    accuracy                           0.74    600000
   macro avg       0.74      0.74      0.74    600000
weighted avg       0.74      0.74      0.74    600000



In [43]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [44]:
submit_df4 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df4['target'] = y_pred_final_dnn4.ravel()
submit_df4.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model4_Submission.csv", index=False)
submit_df4.head()

,id,target
0,600000,0.743290
1,600001,0.724928
2,600002,0.753394
3,600003,0.321530
4,600004,0.690102


# Model5-DNN

In [ ]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model5 = dnn_model()
        model5.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model5.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model5 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model5.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model5.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model5, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn5 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn5 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
y_pred_meta = np.mean(y_pred_meta_dnn5, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

In [ ]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

In [ ]:
submit_df5 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df5['target'] = y_pred_final_dnn5.ravel()
submit_df5.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model4_Submission.csv", index=False)
submit_df5.head()

# EDA-6 luca

Credits to the following beautiful notebook by Bex - https://www.kaggle.com/bextuychiev/model-explainability-with-shap-only-guide-u-need/notebook

I have also used the following one from Luca as reference - https://www.kaggle.com/lucamassaron/feature-selection-by-boruta-shap

Following is a good notebook on LOFO - https://www.kaggle.com/frankmollard/lofo-importance-correlations-tps-nov-21

I am doing a simple SelectKBest (30 variables) in this data and then taking common ones from the above methods, to see what variables truly stand out.

Will update if I add another set and once I try mutual information with my engineered variables

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/test.csv")

In [ ]:
train_df = train[['f1', 'f10', 'f11', 'f14', 'f15', 'f16', 'f17',
                     'f2', 'f20', 'f21', 'f22','f24','f25', 'f26',
                     'f27', 'f28', 'f3', 'f30', 'f31', 'f32', 'f33',
                     'f34', 'f36', 'f37', 'f4', 'f40', 'f41','f42',
                     'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49',
                     'f5','f50', 'f51', 'f53', 'f54', 'f55', 'f57',
                     'f58', 'f59', 'f60', 'f61', 'f62', 'f64', 'f66',
                     'f67', 'f70', 'f71', 'f76','f77', 'f8', 'f80',
                     'f81','f82', 'f83', 'f87', 'f89', 'f9', 'f90',
                     'f91', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98']]
test_df = test[['f1', 'f10', 'f11', 'f14', 'f15', 'f16', 'f17',
                     'f2', 'f20', 'f21', 'f22','f24','f25', 'f26',
                     'f27', 'f28', 'f3', 'f30', 'f31', 'f32', 'f33',
                     'f34', 'f36', 'f37', 'f4', 'f40', 'f41','f42',
                     'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49',
                     'f5','f50', 'f51', 'f53', 'f54', 'f55', 'f57',
                     'f58', 'f59', 'f60', 'f61', 'f62', 'f64', 'f66',
                     'f67', 'f70', 'f71', 'f76','f77', 'f8', 'f80',
                     'f81','f82', 'f83', 'f87', 'f89', 'f9', 'f90',
                     'f91', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98']]

In [ ]:
train_df['mean'] = train_df.mean(axis=1)
train_df['std'] = train_df.std(axis=1)
train_df['var'] = train_df.var(axis=1)
train_df['sum'] = train_df.sum(axis=1)
train_df['max'] = train_df.max(axis=1)
train_df['kurt'] = train_df.kurt(axis=1)
train_df['quantile'] = train_df.quantile(axis=1)


test_df['mean'] = test_df.mean(axis=1)
test_df['std'] = test_df.std(axis=1)
test_df['var'] = test_df.var(axis=1)
test_df['sum'] = test_df.sum(axis=1)
test_df['max'] = test_df.max(axis=1)
test_df['kurt'] = test_df.kurt(axis=1)
test_df['quantile'] = test_df.quantile(axis=1)

In [ ]:
train_df = pd.concat([train_df, train['target']],axis = 1)

In [ ]:
features = test_df.columns.tolist()
print(f"Num features: {len(features)}")

Num features: 77


In [ ]:
train_df[features] = train_df[features].astype('float32')
test_df[features] = test_df[features].astype('float32')
print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


train_df: (600000, 78) 
test_df: (540000, 77)


# Model6-DNN

In [ ]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model6 = dnn_model()
        model6.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model6.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model6 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model6.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model6.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model6, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn6 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn6 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.7424148335192977



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.745809829035371



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7468646138465906



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7456450231540226



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.7483499692681611


Seed: 2025 | Aggregate OOF Score: 0.746001044084044


Aggregate OOF Score: 0.7460071374538991


In [ ]:
y_pred_meta = np.mean(y_pred_meta_dnn6, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.73      0.72      0.72    296394
           1       0.73      0.74      0.73    303606

    accuracy                           0.73    600000
   macro avg       0.73      0.73      0.73    600000
weighted avg       0.73      0.73      0.73    600000



In [ ]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [ ]:
submit_df6 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df6['target'] = y_pred_final_dnn6.ravel()
submit_df6.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model6_Submission.csv", index=False)
submit_df6.head()

,id,target
0,600000,0.742414
1,600001,0.733466
2,600002,0.747426
3,600003,0.606355
4,600004,0.718100


# EDA-7 my_features_f_classif 

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/test.csv")

In [ ]:
train_df = train[['f3','f8','f10','f17','f21','f22','f24','f25','f26','f27','f34',
               'f40','f41','f43','f44','f47','f50','f54','f55','f57','f60','f66',
               'f71','f80','f81','f82','f91','f96','f97','f98']]
test_df = test[['f3','f8','f10','f17','f21','f22','f24','f25','f26','f27','f34',
               'f40','f41','f43','f44','f47','f50','f54','f55','f57','f60','f66',
               'f71','f80','f81','f82','f91','f96','f97','f98']]

In [ ]:
features = test_df.columns.tolist()
len(features)

30

In [ ]:
for col in tqdm(features):
    train_df[col+'_bin'] = train_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)
    test_df[col+'_bin'] = test_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)

print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")
train_df.head()

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
 30%|███       | 9/30 [00:15<00:36,  1.75s/it]


KeyboardInterrupt: ignored

In [ ]:
features = test_df.columns.tolist()
print(f"Num features: {len(features)}")

In [ ]:
train_df[features] = train_df[features].astype('float32')
test_df[features] = test_df[features].astype('float32')
print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

# Model7-DNN

In [ ]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model7 = dnn_model()
        model7.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model7.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model7 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model7.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model7.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model7, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn7 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn7 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
y_pred_meta = np.mean(y_pred_meta_dnn7, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

In [ ]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

In [ ]:
submit_df7 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df7['target'] = y_pred_final_dnn5.ravel()
submit_df7.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/DNN_model7_Submission.csv", index=False)
submit_df7.head()

#EDA-8 lofo_features

In [ ]:
lofo_features = set(['f34','f55','f8','f43','f91','f71','f80','f27','f50','f41','f97','f66','f57',
                'f22','f25','f96','f81','f82','f21','f24','f26','f54','f98','f40','f60','f3','f17',
                'f95','f5','f45'])

# Model8-DNN

#EDA-9 cor_features 

In [ ]:
cor_features = set(['f34', 'f55', 'f43', 'f71', 'f80', 'f91', 'f8', 'f27', 'f97', 'f50', 'f41', 'f57',
                    'f25', 'f22', 'f66', 'f96', 'f81', 'f82', 'f21', 'f40', 'f24', 'f60', 'f98', 'f3',
                    'f54', 'f44', 'f26', 'f47', 'f17', 'f10'])

# Model9-DNN

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/test.csv")

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/test.csv")

In [ ]:
df_train = train.drop(['target', 'id'], axis = 1)

In [ ]:
df_test = test.drop(['id'],axis = 1)

In [ ]:
train_df = train.drop(['target', 'id'], axis = 1)

In [ ]:
test_df = test.drop(['id'],axis = 1)

In [ ]:
features = test_df.columns.tolist()
len(features)

100

In [ ]:
for col in tqdm(features):
    train_df[col+'_bin'] = train_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)
    test_df[col+'_bin'] = test_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0)

print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")
train_df.head()

100%|██████████| 100/100 [02:33<00:00,  1.53s/it]

train_df: (600000, 200) 
test_df: (540000, 200)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f60_bin,f61_bin,f62_bin,f63_bin,f64_bin,f65_bin,f66_bin,f67_bin,f68_bin,f69_bin,f70_bin,f71_bin,f72_bin,f73_bin,f74_bin,f75_bin,f76_bin,f77_bin,f78_bin,f79_bin,f80_bin,f81_bin,f82_bin,f83_bin,f84_bin,f85_bin,f86_bin,f87_bin,f88_bin,f89_bin,f90_bin,f91_bin,f92_bin,f93_bin,f94_bin,f95_bin,f96_bin,f97_bin,f98_bin,f99_bin
0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.806880,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.33055,0.009397,0.144738,3.05131,1.30362,0.033225,-0.018284,2.748210,-0.009294,-0.036271,-0.049871,0.019484,3.898460,11.2863,1.138020,3.366880,4.94446,-0.105772,...,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1
1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.262150,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.60444,0.103441,0.067638,4.75362,1.85552,-0.181834,0.008359,3.166340,0.011850,0.022292,0.069320,0.117109,0.315276,24.4807,1.672270,-0.409067,4.95475,0.092358,...,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1
2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.069490,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.68365,0.038180,0.123716,1.11248,3.57166,0.120601,0.082069,2.233520,0.002270,0.045182,0.014405,0.011599,-0.502849,33.7382,1.417500,1.071350,3.22296,2.122030,...,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1
3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,1.002680,4.869600,0.058411,2.497850,1.23843,2.348360,0.175475,1.608890,2.02881,0.042086,0.005141,0.076506,1.65122,0.111813,0.121641,0.58912,4.23692,-0.032843,0.058168,0.712927,0.097465,0.072744,0.000324,0.063362,4.063820,25.3824,0.576572,2.026210,2.96843,1.085670,...,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,1,1
4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,0.711483,0.769988,0.057555,0.957257,3.71145,5.464350,0.287104,2.616950,1.38403,0.074883,-0.010543,0.109121,2.27602,0.008023,0.045236,4.35954,5.07562,-0.009376,0.528966,4.053350,0.020000,0.106828,0.051307,0.045939,3.402460,15.5615,1.635960,0.047029,4.01771,0.155748,...,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1


In [ ]:
df_train['mean'] = df_train.mean(axis=1)
df_train['std'] = df_train.std(axis=1)
df_train['var'] = df_train.var(axis=1)
df_train['sum'] = df_train.sum(axis=1)
df_train['max'] = df_train.max(axis=1)
df_train['kurt'] = df_train.kurt(axis=1)
df_train['quantile'] = df_train.quantile(axis=1)


df_test['mean'] = df_test.mean(axis=1)
df_test['std'] = df_test.std(axis=1)
df_test['var'] = df_test.var(axis=1)
df_test['sum'] = df_test.sum(axis=1)
df_test['max'] = df_test.max(axis=1)
df_test['kurt'] = df_test.kurt(axis=1)
df_test['quantile'] = df_test.quantile(axis=1)

In [ ]:
df_train = df_train[['mean','std', 'var', 'sum', 'max', 'kurt', 'quantile']]
df_test = df_test[['mean','std', 'var', 'sum', 'max', 'kurt', 'quantile']]

In [ ]:
train_df = pd.concat([train_df,df_train,train['target']],axis = 1)
test_df = pd.concat([test_df,df_test],axis = 1)

In [ ]:
features = test_df.columns.tolist()
print(f"Num features: {len(features)}")

Num features: 207


In [ ]:
train_df[features] = train_df[features].astype('float32')
test_df[features] = test_df[features].astype('float32')
print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

train_df: (600000, 208) 
test_df: (540000, 207)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f67_bin,f68_bin,f69_bin,f70_bin,f71_bin,f72_bin,f73_bin,f74_bin,f75_bin,f76_bin,f77_bin,f78_bin,f79_bin,f80_bin,f81_bin,f82_bin,f83_bin,f84_bin,f85_bin,f86_bin,f87_bin,f88_bin,f89_bin,f90_bin,f91_bin,f92_bin,f93_bin,f94_bin,f95_bin,f96_bin,f97_bin,f98_bin,f99_bin,mean,std,var,sum,max,kurt,quantile
0,0.003229,4.838660,585.528992,2.282910,0.713180,3.907830,0.480696,1.482270,4.891810,0.056351,4.200990,3.151800,0.000349,1.851160,2.63889,0.746668,-0.004756,1.610300,4.11482,-0.077756,0.129446,0.053324,0.416789,0.445009,0.150464,5.021300,2.221390,-0.072333,-0.215874,1.56236,0.074881,0.010050,0.018582,0.067466,5.578300,3.085560,3.842470,0.011125,2.35997,0.695092,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.101993,58.157021,3374.306641,4149.764648,4149.764648,32.237778,0.430899
1,0.008602,0.505536,-100.098999,3.012670,0.027199,1.194610,5.036620,2.517440,4.553890,0.063876,0.337257,4.439690,0.013188,3.379010,3.38470,1.167400,2.246550,1.750170,2.76624,-0.058501,0.012595,0.036144,0.769057,0.017496,0.050283,0.324697,4.948640,0.124789,0.347128,1.24512,0.035822,-0.013188,0.023194,0.006444,4.983330,23.706900,8.287290,4.796230,1.79928,-0.050040,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.551832,10.509767,110.333733,176.578537,176.578537,28.490255,0.394878
2,1.461000,2.437260,-112.963997,3.541230,0.752338,4.338310,1.648080,4.699910,1.950250,0.005303,2.071680,0.546499,0.141781,1.673170,4.30649,1.702330,-0.062869,1.619230,4.19053,0.055140,-0.016590,0.017805,3.064810,0.070370,0.098316,3.507540,1.069100,0.012750,0.009981,3.46781,0.035920,-0.009804,0.065728,-0.004725,5.281020,11.528800,0.171694,4.394570,2.52084,0.079365,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.224444,11.536294,133.022888,167.228073,167.228073,25.425522,0.278908
3,0.140556,3.085610,179.451004,0.573945,0.057342,2.216790,1.623480,0.526174,1.542540,-0.026160,1.609440,1.723560,-0.019564,1.552130,4.83264,1.501640,0.192669,4.614890,1.47069,-0.010031,0.072805,0.048035,3.230210,-0.031548,0.028697,3.752520,4.948470,-0.174542,-0.033491,2.47823,0.068130,0.090797,0.029877,0.146718,3.169830,-12.984600,3.135210,1.765010,3.25399,0.713238,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.714848,17.870302,318.437653,610.507568,610.507568,36.194351,0.182896
4,0.128876,5.199760,107.466003,-0.497149,0.080220,0.458121,0.629839,5.240460,-0.232279,0.030006,0.481359,2.176020,0.193162,1.392090,2.51890,2.993170,4.170910,0.318375,4.84563,0.085064,0.026443,-0.004559,0.120327,-0.008630,0.004495,4.921700,1.685640,0.095628,0.189131,2.56955,0.046643,0.111462,0.002912,0.060737,4.253000,38.703400,0.170825,-0.598784,3.92796,0.262956,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.540107,11.305001,127.290840,395.146667,395.146667,41.474819,0.181556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539995,0.431599,1.507560,24.269800,2.928480,2.788830,5.152330,2.779980,0.816389,4.791560,0.026331,4.562970,0.233379,0.254225,2.213830,5.58360,3.800710,0.503562,4.061380,2.75466,0.112982,-0.038411,0.077714,3.010950,-0.008242,0.043315,0.080434,2.371080,0.133727,0.355986,2.01470,0.002188,0.220446,0.078856,0.084574,0.758547,-3.627310,2.367110,0.179322,1

# Model10-DNN

In [ ]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 512

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df[features], train_df['target'])):
        counter += 1

        train_x, train_y = train_df[features].iloc[train], train_df['target'].iloc[train]
        val_x, val_y = train_df[features].iloc[val], train_df['target'].iloc[val]

        model7 = dnn_model()
        model7.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                               patience=4, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=15, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model7.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=300,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        model7 = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model7.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model7.predict(test_df, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model7, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn7 = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn7 = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-0 | OOF Score: 0.744060517545417



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2021 | Fold-1 | OOF Score: 0.7488193638908178



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


...


Seed-2025 | Fold-2 | OOF Score: 0.7494473348477763



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-3 | OOF Score: 0.7485224330972622



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Seed-2025 | Fold-4 | OOF Score: 0.7518934219473795


Seed: 2025 | Aggregate OOF Score: 0.7491112539832844


Aggregate OOF Score: 0.7483860560624487


In [ ]:
y_pred_meta = np.mean(y_pred_meta_dnn7, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(train_df['target'], y_pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73    296394
           1       0.74      0.74      0.74    303606

    accuracy                           0.73    600000
   macro avg       0.73      0.73      0.73    600000
weighted avg       0.73      0.73      0.73    600000



In [ ]:
cnf_matrix = confusion_matrix(train_df['target'], y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [ ]:
submit_df7 = pd.read_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df7['target'] = y_pred_final_dnn7.ravel()
submit_df7.to_csv("/content/drive/MyDrive/AI/dataset/tabular-playground-series-nov-2021/Derived_data.csv", index=False)
submit_df7.head()

,id,target
0,600000,0.723546
1,600001,0.732156
2,600002,0.739589
3,600003,0.479302
4,600004,0.721516
